In [1]:
from story_sage import *
import os
import glob
from story_sage.utils.raptor import _RaptorResults
from pprint import pprint
import logging
import markdown
from IPython.display import HTML

logging.basicConfig(level=logging.WARN)

logger = logging.getLogger()
logger.addHandler(logging.StreamHandler())

os.environ['TOKENIZERS_PARALLELISM'] = "false"

config_path = './config.yml'
ssconfig = StorySageConfig.from_file(config_path)
selected_series = next(series for series in ssconfig.series if series.series_metadata_name == 'wheel_of_time')

processed_file_name = './chunks/wheel_of_time/raptor_chunks/*.json.gz'

retriever = StorySageRetriever(chroma_path=ssconfig.chroma_path, chroma_collection_name=ssconfig.raptor_collection, n_chunks=15)

for file_name in glob.iglob(processed_file_name):
    print(f"Processing {file_name}")
    #retriever.load_processed_files(file_name, selected_series.series_id)

Processing ./chunks/wheel_of_time/raptor_chunks/wheel_of_time_06_07.json.gz
Processing ./chunks/wheel_of_time/raptor_chunks/wheel_of_time_11.json.gz
Processing ./chunks/wheel_of_time/raptor_chunks/wheel_of_time_01.json.gz
Processing ./chunks/wheel_of_time/raptor_chunks/wheel_of_time_10.json.gz
Processing ./chunks/wheel_of_time/raptor_chunks/wheel_of_time_04_05.json.gz
Processing ./chunks/wheel_of_time/raptor_chunks/wheel_of_time_12.json.gz
Processing ./chunks/wheel_of_time/raptor_chunks/wheel_of_time_02_03.json.gz
Processing ./chunks/wheel_of_time/raptor_chunks/wheel_of_time_13.json.gz
Processing ./chunks/wheel_of_time/raptor_chunks/wheel_of_time_9.json.gz
Processing ./chunks/wheel_of_time/raptor_chunks/wheel_of_time_8.json.gz


In [4]:
conversation = StorySageConversation()

state = StorySageState(
    question=None,
    book_number=11,
    chapter_number=1,
    series_id=selected_series.series_id,
    conversation=conversation,
)
chain = StorySageChain(config=ssconfig, state=state, log_level=logging.WARN)

def ask_question(question: str):
    chain.state.question = question
    result = chain.invoke()
    conversation.add_turn(question=question, response=result.answer, context=result.context)
    chain.state.conversation = conversation
    return HTML(f'<div style="max-width:1000px;">{markdown.markdown(result.answer)}</div>')

In [5]:
ask_question("Tell me about the philosophy of the tua'ath'an")

In [6]:
ask_question("no, just tell me about what they believe in and how it influences their behavior")

In [7]:
ask_question("do they eat meat?")

In [11]:
pprint(chain.state.conversation.__str__())

("HUMAN: Tell me about the philosophy of the tua'ath'an\n"
 'COMPUTER: Are there specific chapters or passages in the book series that '
 "mention the Tuatha'an or their philosophy that you would like me to look "
 'at?\n'
 'HUMAN: no, just tell me about what they believe in and how it influences '
 'their behavior\n'
 "COMPUTER: The Tuatha'an, also known as the Traveling People or Tinkers, "
 'adhere to a philosophy that emphasizes non-violence and the belief that one '
 'can oppose evil without resorting to harm. This philosophy is encapsulated '
 'in their Way of the Leaf, which teaches that violence harms both the '
 'perpetrator and the victim. Here are some key points about their beliefs and '
 'how these influence their behavior:\n'
 '\n'
 "- **Non-Violence**: The Tuatha'an believe that it is better to flee from "
 'those who would harm them rather than to defend themselves with violence. '
 'This belief stems from the idea that engaging in violence would make them no '
 'differ